In [1]:
# test

In [2]:
import paramiko
import zipfile
import pickle
import os
import pandas as pd
from datetime import datetime, timedelta

def download_and_extract_all_files(hostname, port, username, password, remote_folder, local_folder):
    # Create an SSH client
    ssh = paramiko.SSHClient()

    try:
        # Automatically add the server's host key
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Connect to the SSH server
        ssh.connect(hostname, port, username, password)

        # Open an SFTP session
        with ssh.open_sftp() as sftp:
            # Change to the remote folder
            sftp.chdir(remote_folder)

            # List files in the remote folder
            files = sftp.listdir()

            # Dictionary to store dataframes for each date
            date_dataframes = {}

            # Iterate through files and download/extract for each date
            for file in files:
                if file.startswith("UnderlyingEOD") and not "Summaries" in file:
                    date_format = file.split("_")[-1].split(".")[0]  # Extract date from the filename
                    remote_path = os.path.join(remote_folder, file).replace("\\", "/")
                    local_path = os.path.join(local_folder, file).replace("\\", "/")

                    # Download the file
                    sftp.get(remote_path, local_path)

                    # Extract files from the downloaded ZIP file
                    extract_zip(local_path, local_folder)

                    # Read the CSV file into a DataFrame
                    csv_file = [csv for csv in os.listdir(local_folder) if csv.endswith('.csv')]
                    if csv_file:
                        csv_file_path = os.path.join(local_folder, csv_file[0])
                        eod_df = pd.read_csv(csv_file_path)

                        # Store the DataFrame in the dictionary with the date as the key
                        date_dataframes[date_format] = eod_df

            # Clean up: Remove downloaded ZIP files
            for file in os.listdir(local_folder):
                file_path = os.path.join(local_folder, file)
                if file_path.endswith(".zip"):
                    os.remove(file_path)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close the SSH connection
        ssh.close()

    return date_dataframes

def extract_zip(zip_filename, extraction_folder):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # Extract all files in the ZIP archive to the extraction folder
        zip_ref.extractall(extraction_folder)

# SFTP server details
sftp_hostname = "sftp.datashop.livevol.com"
sftp_port = 22  # Change the port if your SFTP server uses a different port
sftp_username = "nan2_lehigh_edu"
sftp_password = "PAIndex2023!"
sftp_remote_folder = "/subscriptions/order_000046197/item_000053507"  # Change to the actual path on the SFTP server
local_download_folder = os.path.join(os.getcwd(), "input")  # Set to "input" folder inside the current directory

# Create the local download folder if it doesn't exist
os.makedirs(local_download_folder, exist_ok=True)

# Download all files and extract for each date
date_dataframes = download_and_extract_all_files(sftp_hostname, sftp_port, sftp_username, sftp_password, sftp_remote_folder, local_download_folder)

C:\Users\micha\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
local_download_folder = os.path.join(os.getcwd(), "input")  # Set to "input" folder inside the current directory

# Create a dictionary to store dataframes for each date
date_dataframes = {}

# Loop through all files in the "input" folder
for file in os.listdir(local_download_folder):
    if file.startswith("UnderlyingEOD_") and file.endswith(".csv"):
        # Extract date from the filename
        date_format = file.split("_")[-1].split(".")[0]

        # Create the full file path
        csv_file_path = os.path.join(local_download_folder, file)

        # Read the CSV file into a DataFrame
        eod_df = pd.read_csv(csv_file_path)

        # Store the DataFrame in the dictionary with the date as the key
        date_dataframes[date_format] = eod_df

        # After using the CSV file, optionally delete it
        try:
            os.remove(csv_file_path)
            print(f"Deleted: {csv_file_path}")
        except FileNotFoundError:
            print(f"File not found: {csv_file_path}")
        except Exception as e:
            print(f"Error deleting file: {e}")

# Now, you can access each date's DataFrame using date_dataframes[date_format]
date_dataframes.pop(date_format)
output_file_path = os.path.join(os.getcwd(), r"input\date_dataframes.pkl")
with open(output_file_path, 'wb') as output_file:
    pickle.dump(date_dataframes, output_file)

print(f"Saved date_dataframes to: {output_file_path}")

Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-05-24.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-05-28.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-05-29.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-05-30.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-05-31.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-06-03.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-06-04.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-06-05.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-06-06.csv
Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\Underlyin

In [4]:
#date_dataframes['2023-11-21']

In [5]:
df = pd.read_excel("input/RAY as of Oct 23 20231_PA.xlsx")
df.columns = df.columns.str.rstrip('\n')
df['Ticker'] = df['Ticker'].str.split(' ', n=1, expand=True)[0]
df['Ticker'] = df['Ticker'].str.replace(' ', '')  # Remove spaces
# Create float_df by selecting the first 100 rows of "Ticker" and "Equity Float" columns
float_df = df[['Ticker', 'Equity Float']].head(100)
float_df

,Ticker,Equity Float
0,CMCSA,4085497600
1,FIVE,54404088
2,DKS,59332580
3,TOL,106831880
4,ARMK,259611840
...,...,...
95,PPG,235286400
96,FMC,123947608
97,X,198392064
98,AXTA,220914096


In [6]:
# Create an empty DataFrame to store market cap DataFrames for each date
eod_market_cap_pivot = pd.DataFrame()

# Extract unique dates from date_dataframes dictionary
unique_dates = list(date_dataframes.keys())

# Iterate over unique dates
for date_format in unique_dates[:-1]:
    # Access the corresponding DataFrame from date_dataframes
    eod_df = date_dataframes[date_format]

    # Merge dataframes on the common column 'Ticker' and 'underlying_symbol'
    merged_df = pd.merge(float_df, eod_df, left_on='Ticker', right_on='underlying_symbol')

    # Perform the multiplication and rename the column to 'Market Cap'
    merged_df['Market Cap'] = merged_df['Equity Float'] * merged_df['close']

    # Use pivot_table to create a multi-level DataFrame
    eod_market_cap_daily = merged_df.pivot_table(index='quote_date', columns='Ticker', values='Market Cap', aggfunc='sum')

    # Add a new column for the sum of market caps for each date
    #eod_market_cap_daily['close index market cap'] = eod_market_cap_daily.sum(axis=1)

    # Append the daily market cap DataFrame to the main DataFrame
    eod_market_cap_pivot = pd.concat([eod_market_cap_pivot, eod_market_cap_daily])

# Rename the index to 'Date'
eod_market_cap_pivot = eod_market_cap_pivot.rename_axis(index='Date')

# Display the resulting pivot table
eod_market_cap_pivot.index = pd.to_datetime(eod_market_cap_pivot.index, errors='coerce')
#eod_market_cap_pivot.index.name = None  # Remove the index name

# Display the resulting pivot table
eod_market_cap_pivot
eod_market_cap_pivot.to_csv("input/LUPA100Mktcap_2023.csv")

In [7]:
eod_market_cap_pivot_new = pd.read_csv("input/LUPA100Mktcap_2023.csv", index_col = "Date")
eod_market_cap_pivot_new

,AA,ACNB,ACRS,AEO,AHCO,AME,ANSS,APD,ARMK,AUR,...,VPG,VRCA,VSH,VSTS,VTRS,WAB,WCC,WMK,WST,X
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-24,7.354565e+09,2.593273e+08,71674525.44,4.197110e+09,7.924148e+08,3.988995e+10,2.862515e+10,5.855851e+10,8.297194e+09,1.726650e+09,...,3.848523e+08,2.097464e+08,3.019246e+09,1.565892e+09,1.243067e+10,3.009584e+10,9.659697e+09,1.071050e+09,2.436594e+10,7.122275e+09
2024-05-28,7.631862e+09,2.625995e+08,67194867.60,4.328354e+09,7.947732e+08,3.867254e+10,2.835554e+10,5.861387e+10,8.260849e+09,1.691698e+09,...,3.845018e+08,2.072904e+08,2.996602e+09,1.512480e+09,1.240677e+10,2.992027e+10,9.375366e+09,1.051814e+09,2.405883e+10,7.304796e+09
2024-05-29,7.759030e+09,2.573639e+08,65275014.24,4.350077e+09,7.853397e+08,3.832339e+10,2.786394e+10,5.763517e+10,8.081717e+09,1.698689e+09,...,3.801789e+08,2.072904e+08,2.942507e+09,1.517691e+09,1.232908e+10,2.970745e+10,8.929930e+09,1.052140e+09,2.370470e+10,7.364313e+09
2024-05-30,7.866770e+09,2.620269e+08,67194867.60,4.024229e+09,7.617559e+08,3.851864e+10,2.764609e+10,5.767945e+10,8.151812e+09,1.677717e+09,...,3.871890e+08,2.048344e+08,2.941249e+09,1.578920e+09,1.252629e+10,2.998411e+10,9.027906e+09,1.060454e+09,2.418814e+10,7.562705e+09
2024-05-31,7.822615e+09,2.567095e+08,65594989.80,3.977162e+09,7.444612e+08,3.899642e+10,2.746549e+10,5.907444e+10,8.343925e+09,1.667231e+09,...,3.939654e+08,2.095008e+08,2.972700e+09,1.604975e+09,1.266972e+10,3.001958e+10,9.061237e+09,1.068605e+09,2.434757e+10,7.606352e+09
2024-06-03,7.772277e+09,2.531918e+08,65914965.36,4.020608e+09,7.515363e+08,3.817868e+10,2.719847e+10,6.001550e+10,8.323156e+09,1.579850e+09,...,3.945495e+08,2.104833e+08,3.003521e+09,1.621259e+09,1.265179e+10,2.939177e+10,8.948111e+09,1.074963e+09,2.416463e+10,7.683725e+09
2024-06-04,7.418149e+09,2.485288e+08,65275014.24,3.910182e+09,7.295248e+08,3.833947e+10,2.739808e+10,5.985608e+10,8.307579e+09,1.478488e+09,...,3.842681e+08,2.040975e+08,2.879606e+09,1.610186e+09,1.257410e+10,2.921974e+10,8.719333e+09,1.047575e+09,2.339392e+10,7.556754e+09
2024-06-05,7.522356e+09,2.552369e+08,67194867.60,3.924664e+09,8.120680e+08,3.874604e+10,2.827258e+10,5.985608e+10,8.276425e+09,1.628784e+09,...,3.897593e+08,2.242371e+08,2.933701e+09,1.631029e+09,1.255020e+10,2.938468e+10,8.983968e+09,1.033229e+09,2.333147e+10,7.612303e+09
2024-06-06,7.532954e+09,2.511875e+08,64635063.12,3.901131e+09,8.745650e+08,3.876671e+10,2.807470e+10,5.974093e+10,8.484115e+09,1.625288e+09,...,3.862543e+08,2.161322e+08,2.904767e+09,1.619305e+09,1.244860e+10,2.909560e+10,8.861751e+09,1.043500e+09,2.315734e+10,7.602384e+09


In [4]:
# Create an empty DataFrame to store market cap DataFrames for each date
eod_market_cap_pivot = pd.DataFrame()

# Extract unique dates from date_dataframes dictionary
unique_dates = list(date_dataframes.keys())

# Iterate over unique dates
for date_format in unique_dates:
    # Access the corresponding DataFrame from date_dataframes
    eod_df = date_dataframes[date_format]

    # Merge dataframes on the common column 'Ticker' and 'underlying_symbol'
    merged_df = pd.merge(float_df, eod_df, left_on='Ticker', right_on='underlying_symbol')

    # Perform the multiplication and rename the column to 'Market Cap'
    merged_df['Market Cap'] = merged_df['Equity Float'] * merged_df['close']

    # Use pivot_table to create a multi-level DataFrame
    eod_market_cap_daily = merged_df.pivot_table(index='quote_date', columns='Ticker', values='Market Cap', aggfunc='sum')

    # Add a new column for the sum of market caps for each date
    eod_market_cap_daily['close index market cap'] = eod_market_cap_daily.sum(axis=1)

    # Append the daily market cap DataFrame to the main DataFrame
    eod_market_cap_pivot = pd.concat([eod_market_cap_pivot, eod_market_cap_daily])

# Rename the index to 'Date'
eod_market_cap_pivot = eod_market_cap_pivot.rename_axis(index='Date')

# Display the resulting pivot table
eod_market_cap_pivot.index = pd.to_datetime(eod_market_cap_pivot.index, errors='coerce')
#eod_market_cap_pivot.index.name = None  # Remove the index name

# Display the resulting pivot table
eod_market_cap_pivot

KeyError: 'underlying_symbol'

In [8]:
eod_market_cap_pivot["CMCSA"]
#eod_market_cap_pivot[["CMCSA"]]

Date
2023-11-10    1.711415e+11
2023-11-13    1.694256e+11
2023-11-14    1.710598e+11
2023-11-15    1.737562e+11
2023-11-16    1.733068e+11
2023-11-17    1.733068e+11
2023-11-20    1.754925e+11
2023-11-21    1.740013e+11
2023-11-22    1.734702e+11
2023-11-24    1.740422e+11
2023-11-27    1.718769e+11
2023-11-28    1.710598e+11
2023-11-29    1.700740e+11
2023-11-30    1.711823e+11
2023-12-01    1.724080e+11
2023-12-04    1.759215e+11
2023-12-05    1.700793e+11
2023-12-06    1.704061e+11
2023-12-07    1.722037e+11
2023-12-08    1.718769e+11
Name: CMCSA, dtype: float64

In [9]:
# # Create the new columns
# eod_market_cap_pivot['gross_change_mcap'] = eod_market_cap_pivot['close index market cap'].diff().fillna(0)

# # Calculate mkt_cap_deleted_stock
# eod_market_cap_pivot['mkt_cap_deleted_stock'] = eod_market_cap_pivot.apply(
#     lambda row: row['close index market cap'] if pd.isna(row['close index market cap']) or row['close index market cap'] == 0 else 0,
#     axis=1
# )

# # Fill NaNs with 0 in the mkt_cap_deleted_stock column
# eod_market_cap_pivot['mkt_cap_deleted_stock'] = eod_market_cap_pivot['mkt_cap_deleted_stock'].fillna(0)

# # Calculate adjusted_mkt_cap
# eod_market_cap_pivot['adjusted_mkt_cap'] = eod_market_cap_pivot['close index market cap'].shift(1) - eod_market_cap_pivot['mkt_cap_deleted_stock']

# # Fill NaNs with 0 in the adjusted_mkt_cap column
# eod_market_cap_pivot['adjusted_mkt_cap'] = eod_market_cap_pivot['adjusted_mkt_cap'].fillna(0)

# # Calculate divisor
# eod_market_cap_pivot['divisor'] = eod_market_cap_pivot.apply(
#     lambda row: 1.00 if pd.isna(row['mkt_cap_deleted_stock']) or row['mkt_cap_deleted_stock'] == 0 else row['adjusted_mkt_cap'] / row['close index market cap'],
#     axis=1
# )

# # Calculate gross index level
# eod_market_cap_pivot['gross index level'] = eod_market_cap_pivot['close index market cap'] / eod_market_cap_pivot['divisor']

# # Calculate Index Value
# eod_market_cap_pivot['Index Value'] = eod_market_cap_pivot['gross index level'] / eod_market_cap_pivot['gross index level'].iloc[0] * 100

# # Display the DataFrame with the new columns
# eod_market_cap_pivot


In [10]:
# # Create the new columns
# eod_market_cap_pivot['gross_change_mcap'] = eod_market_cap_pivot['close index market cap'].diff().fillna(0)
# # Calculate mkt_cap_deleted_stock
# eod_market_cap_pivot['mkt_cap_deleted_stock'] = eod_market_cap_pivot.apply(
#     lambda row: row['close index market cap'] if pd.isna(row['close index market cap']) or row['close index market cap'] == 0 else 0,
#     axis=1
# )

# eod_market_cap_pivot['mkt_cap_deleted_stock'] = mkt_cap_deleted_stock.fillna(0)

# # Calculate adjusted_mkt_cap
# eod_market_cap_pivot['adjusted_mkt_cap'] = eod_market_cap_pivot['close index market cap'].shift(1) - eod_market_cap_pivot['mkt_cap_deleted_stock']
# # Calculate divisor
# eod_market_cap_pivot['divisor'] = eod_market_cap_pivot.apply(
#     lambda row: 1.00 if pd.isna(row['adjusted_mkt_cap']) or row['adjusted_mkt_cap'] == 0 else row['adjusted_mkt_cap'] / row['close index market cap'],
#     axis=1
# )
# # Calculate gross index level
# eod_market_cap_pivot['gross index level'] = eod_market_cap_pivot['close index market cap'] / eod_market_cap_pivot['divisor']

# # Calculate Index Value
# eod_market_cap_pivot['Index Value'] = eod_market_cap_pivot['gross index level'] / eod_market_cap_pivot['gross index level'].iloc[0] * 100

# # Display the DataFrame with the new columns
# eod_market_cap_pivot


In [1]:
float_df

NameError: name 'float_df' is not defined